In [1]:
%pylab inline
from sklearn import linear_model
import cv2

Populating the interactive namespace from numpy and matplotlib


In [2]:
max_iter = 10000

# "noise must be a scalar for the entire image"
sigma_r = 100
sigma_n = 100
epsilon = 0.01
beta_epsilon = 0.001

In [4]:
R = cv2.imread('./Images/RealImage.jpg', cv2.CV_LOAD_IMAGE_GRAYSCALE) 
N = cv2.imread('./Images/NewImage.jpg', cv2.CV_LOAD_IMAGE_GRAYSCALE) 

mean = 0
row_r, col_r = R.shape
row_n, col_n = N.shape
# We are not how to represent the PSF used. 
PSF_r = np.random.normal(mean, 3, (row_r,col_r))
PSF_n = np.random.normal(mean, 3, (row_n,col_n))

In [10]:
fourier_R = fft.fft2(R)
fourier_N = fft.fft2(N)
fourier_PSF_r = fft.fft2(PSF_r)
fourier_PSF_n = fft.fft2(PSF_n)

In [6]:
#This is not a robust fit, which the article calls for
def linear_fit(D_r, D_n):
    D_r_elements = D_r.flatten().reshape(-1, 1) 
    D_n_elements = D_n.flatten().reshape(-1, 1) 
    lr_model = linear_model.LinearRegression().fit(D_r_elements, D_n_elements)
    slope = lr_model.coef_[0]
    intercept = lr_model.intercept_
    return slope, intercept

In [12]:
beta = 1
gamma_prime = 0

beta_converges = False
num_iter = 0

while not beta_converges and num_iter < max_iter:
    num_iter = num_iter + 1
    denom = (beta**2)*(sigma_r**2)*(fourier_PSF_n*conjugate(fourier_PSF_n)) + (sigma_n**2)*(fourier_PSF_r*conjugate(fourier_PSF_r)) + epsilon
    fourier_D_n = (fourier_PSF_r*fourier_N)/sqrt(denom)
    fourier_D_r = (fourier_PSF_n*fourier_R)/sqrt(denom)
    
    D_n = ifft2(fourier_D_n)
    D_r = ifft2(fourier_D_r)
    
    # here, they select pixels which are one (really three) sigma above the image noise
    # We don't do that
    
    prev_beta = beta
    # the fit we are using ignores the imaginary part...
    beta, gamma_prime = linear_fit(real(D_r), real(D_n))
    
    if abs(beta - prev_beta) < beta_epsilon:
        beta_converges = True
gamma = gamma_prime*sqrt(sigma_n**2 + (beta**2)*(sigma_r**2))
print('Number of iterations: ' + str(num_iter))
print('Beta: ' + str(beta))
print('Gamma: ' + str(gamma))

Number of iterations: 3
Beta: [-0.00477517]
Gamma: [ 40.84543608]
